In [13]:
from ultralytics import YOLO
import cv2
import numpy as np
import math
import os
import time
from roboflow import Roboflow

In [3]:
from roboflow import Roboflow
rf = Roboflow(api_key="ejFkGPgDGCQYas0cD5DS")
project = rf.workspace("fpkkgunakanhelmdataset").project("dataset_helm_classification-9wb1c")
version = project.version(2)
dataset = version.download("yolov8")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Dataset_Helm_Classification-2 in yolov8:: 100%|██████████| 9071/9071 [00:05<00:00, 1636.05it/s]


In [3]:
model = YOLO("yolov8n.pt")
# Tambahkan r di depan "..." dan arahkan ke data.yaml
data_path = r"C:\Users\Royan\Documents\Semester 3\komputcerdas\FP\Dataset_Helm_Classification-2\data.yaml"
model.train(data=data_path, epochs=50, imgsz=640, batch=16)

Ultralytics 8.3.233  Python-3.10.0 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 6GB Laptop GPU, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\Royan\Documents\Semester 3\komputcerdas\FP\Dataset_Helm_Classification-2\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001393ED906A0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          

In [16]:
# ==============================================================================
# 1. KONFIGURASI & TUNING AREA (UBAH DISINI AJA)
# ==============================================================================

# --- PATH ---
VIDEO_PATH = r"test\14813939_1080_1920_30fps.mp4"       
MODEL_PATH = r"runs/detect/train2/weights/best.pt" 
TARGET_WIDTH = 360  # Ukuran jendela tampilan (kecilkan jika layar laptop sempit)

# --- SETTING DETEKSI (YOLO) ---
CONF_THRESHOLD = 0.25    # Minimal keyakinan model (0.0 - 1.0)
HELMET_DIST_THRESHOLD = 250 # Jarak (pixel) maksimal kepala dianggap milik motor tersebut

# --- SETTING PSO (FISIKA PARTIKEL) ---
PSO_N_PARTICLES = 30     # Jumlah partikel (kurangi jadi 20 jika berat/lag)
PSO_W  = 0.4             # INERSIA (0.0-1.0): Makin kecil = makin cepat ngerem/kalem
PSO_C1 = 0.8             # PERSONAL (0.0-2.0): Tarikan ke pengalaman sendiri
PSO_C2 = 0.8             # SOCIAL (0.0-2.0): Tarikan ke posisi terbaik global
PSO_V_MAX = 10           # SPEED LIMIT: Maksimal gerak pixel per frame (biar ga teleport)
INIT_SPREAD = 0.2        # SEBARAN AWAL (0.1-0.5): 0.2 artinya 20% dari lebar kotak

# --- SETTING SMOOTHING (ANTI-GETAR) ---
# Nilai 1.0 = Percaya 100% deteksi baru (sangat getar)
# Nilai 0.1 = Percaya 10% deteksi baru, 90% posisi lama (sangat smooth tapi delay)
SMOOTH_FACTOR_YOLO = 0.7 # Saat YOLO mendeteksi (Normal: 0.6 - 0.8)
SMOOTH_FACTOR_PSO  = 0.4 # Saat YOLO hilang & pakai PSO (Normal: 0.3 - 0.5)

# --- SETTING SPEEDOMETER ---
SPEED_MIN_THRESHOLD = 2.0 # Jika gerak < 2 pixel, anggap speed 0 (biar angka ga flicker saat diam)

# ==============================================================================
# 2. HELPER FUNCTIONS
# ==============================================================================
def resize_display(frame, target_width=TARGET_WIDTH):
    h, w = frame.shape[:2]
    if w < target_width: return frame
    scale = target_width / float(w)
    new_dim = (target_width, int(h * scale))
    return cv2.resize(frame, new_dim, interpolation=cv2.INTER_AREA)

class MotorTracker:
    def __init__(self, track_id, bbox, frame):
        self.id = track_id
        self.bbox = bbox 
        self.prev_center = ((bbox[0]+bbox[2])//2, (bbox[1]+bbox[3])//2)
        self.speed = 0
        self.missing_frames = 0 
        
        self.target_hist = self.get_hist(frame, bbox)
        
        # Init Partikel
        self.particles = self.init_particles(bbox)
        self.v = np.zeros_like(self.particles)
        self.p_best = self.particles.copy()
        self.g_best = self.particles[0]
        
        self.helmet_status = "Unknown"
        self.color = (255, 255, 0)
        
    def get_hist(self, frame, bbox):
        x1, y1, x2, y2 = map(int, bbox)
        h, w, _ = frame.shape
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(w, x2), min(h, y2)
        roi = frame[y1:y2, x1:x2]
        if roi.size == 0: return None
        hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
        hist = cv2.calcHist([hsv], [0, 1], None, [180, 256], [0, 180, 0, 256])
        cv2.normalize(hist, hist)
        return hist.flatten()

    def init_particles(self, bbox, n=PSO_N_PARTICLES):
        particles = []
        x1, y1, x2, y2 = bbox
        w, h = x2-x1, y2-y1
        cx, cy = x1 + w/2, y1 + h/2
        
        spread_x = int(w * INIT_SPREAD)
        spread_y = int(h * INIT_SPREAD)
        
        for _ in range(n):
            dx = np.random.randint(-spread_x, spread_x)
            dy = np.random.randint(-spread_y, spread_y)
            
            scale = np.random.uniform(0.95, 1.05) 
            pw, ph = w * scale, h * scale
            
            px1 = cx + dx - pw/2
            py1 = cy + dy - ph/2
            particles.append([px1, py1, px1+pw, py1+ph])
            
        return np.array(particles, dtype=float)

    def fitness(self, frame, bbox):
        x1, y1, x2, y2 = map(int, bbox)
        h, w, _ = frame.shape
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(w, x2), min(h, y2)
        roi = frame[y1:y2, x1:x2]
        if roi.size == 0 or self.target_hist is None: return 0
        
        curr_hist = self.get_hist(frame, (x1, y1, x2, y2))
        if curr_hist is None: return 0
        return cv2.compareHist(self.target_hist, curr_hist, cv2.HISTCMP_CORREL)

    def update(self, frame, yolo_bbox=None):
        # --- PSO LOGIC ---
        fitness_scores = [self.fitness(frame, p) for p in self.particles]
        best_idx = np.argmax(fitness_scores)
        
        if fitness_scores[best_idx] > self.fitness(frame, self.g_best):
            self.g_best = self.particles[best_idx]
            
        p_best_scores = [self.fitness(frame, p) for p in self.p_best]
        mask = np.array(fitness_scores) > np.array(p_best_scores)
        self.p_best[mask] = self.particles[mask]
        
        r1 = np.random.rand(len(self.particles), 4)
        r2 = np.random.rand(len(self.particles), 4)
        
        # Update Velocity (Pakai Variabel Global)
        self.v = PSO_W * self.v + PSO_C1 * r1 * (self.p_best - self.particles) + PSO_C2 * r2 * (self.g_best - self.particles)
        
        # Velocity Clamping
        self.v = np.clip(self.v, -PSO_V_MAX, PSO_V_MAX)
        
        self.particles += self.v

        # --- SMOOTHING LOGIC ---
        if yolo_bbox is not None:
            # Mode Koreksi (Ada YOLO)
            target_box = np.array(yolo_bbox)
            self.bbox = SMOOTH_FACTOR_YOLO * target_box + (1 - SMOOTH_FACTOR_YOLO) * np.array(self.bbox)
            self.missing_frames = 0
            
            # Reset Partikel jika terlalu jauh
            yx1, yy1, yx2, yy2 = yolo_bbox
            ycx, ycy = (yx1+yx2)/2, (yy1+yy2)/2
            for i in range(len(self.particles)):
                px1, py1, px2, py2 = self.particles[i]
                pcx, pcy = (px1+px2)/2, (py1+py2)/2
                if math.dist((ycx, ycy), (pcx, pcy)) > 100:
                    w_box = yx2 - yx1
                    dx = np.random.randint(-20, 20)
                    dy = np.random.randint(-20, 20)
                    self.particles[i] = [yx1+dx, yy1+dy, yx2+dx, yy2+dy]
        else:
            # Mode Tracking (Murni PSO)
            pso_target = np.array(self.g_best)
            self.bbox = SMOOTH_FACTOR_PSO * pso_target + (1 - SMOOTH_FACTOR_PSO) * np.array(self.bbox)
            self.missing_frames += 1

        self.bbox = tuple(map(int, self.bbox))

        # --- SPEED CALC ---
        curr_center = ((self.bbox[0]+self.bbox[2])//2, (self.bbox[1]+self.bbox[3])//2)
        inst_speed = math.dist(self.prev_center, curr_center)
        
        if inst_speed < SPEED_MIN_THRESHOLD:
            inst_speed = 0
            
        self.speed = inst_speed
        self.prev_center = curr_center

def match_detections_to_trackers(detections, trackers):
    matches = [] 
    unmatched_detections = []
    used_trackers = set()
    
    for i, det in enumerate(detections):
        dx1, dy1, dx2, dy2 = det
        d_center = ((dx1+dx2)//2, (dy1+dy2)//2)
        best_t_idx = -1
        min_dist = 100 
        
        for j, trk in enumerate(trackers):
            if j in used_trackers: continue
            tx1, ty1, tx2, ty2 = trk.bbox
            t_center = ((tx1+tx2)//2, (ty1+ty2)//2)
            dist = math.dist(d_center, t_center)
            if dist < min_dist:
                min_dist = dist
                best_t_idx = j
        
        if best_t_idx != -1:
            matches.append((best_t_idx, i))
            used_trackers.add(best_t_idx)
        else:
            unmatched_detections.append(i)
            
    return matches, unmatched_detections, used_trackers

def check_helmet_status(motor_box, all_yolo_boxes, class_names):
    mx1, my1, mx2, my2 = motor_box
    mc = ((mx1+mx2)//2, (my1+my2)//2)
    final_status = "Unknown"
    final_color = (0, 255, 255) 
    
    for box in all_yolo_boxes:
        cls_id = int(box.cls[0])
        label = class_names[cls_id]
        bx1, by1, bx2, by2 = map(int, box.xyxy[0])
        hc = ((bx1+bx2)//2, (by1+by2)//2)
        
        if math.dist(mc, hc) < HELMET_DIST_THRESHOLD: 
            if label in ['Helmet', 'helmet']:
                return "PAKAI HELM", (0, 255, 0)
            elif label in ['No_helmet', 'no_helmet', 'head']:
                return "TIDAK PAKAI HELM", (0, 0, 255)
                
    return final_status, final_color

# ==============================================================================
# 3. MAIN PROGRAM
# ==============================================================================
def main():
    print("Loading Model...")
    model = YOLO(MODEL_PATH)
    cap = cv2.VideoCapture(VIDEO_PATH)
    
    trackers = [] 
    next_id = 1
    
    while True:
        ret, frame = cap.read()
        if not ret: break
        
        results = model(frame, conf=CONF_THRESHOLD, verbose=False)[0]
        
        motor_detections = []
        for r in results.boxes:
            label = model.names[int(r.cls[0])]
            if label in ['Motor']:
                motor_detections.append(tuple(map(int, r.xyxy[0])))
        
        matches, unmatched_dets, used_tracker_indices = match_detections_to_trackers(motor_detections, trackers)
        
        # UPDATE MATCHED
        for t_idx, d_idx in matches:
            trackers[t_idx].update(frame, yolo_bbox=motor_detections[d_idx])
            status, color = check_helmet_status(motor_detections[d_idx], results.boxes, model.names)
            if status != "Unknown": 
                trackers[t_idx].helmet_status = status
                trackers[t_idx].color = color

        # UPDATE UNMATCHED (PURE PSO)
        for i in range(len(trackers)):
            if i not in used_tracker_indices:
                trackers[i].update(frame, yolo_bbox=None) 
        
        # CREATE NEW
        for d_idx in unmatched_dets:
            new_trk = MotorTracker(next_id, motor_detections[d_idx], frame)
            status, color = check_helmet_status(motor_detections[d_idx], results.boxes, model.names)
            new_trk.helmet_status = status
            new_trk.color = color
            trackers.append(new_trk)
            next_id += 1
            
        # DELETE OLD
        trackers = [t for t in trackers if t.missing_frames < 30]

        # DRAW
        for t in trackers:
            x1, y1, x2, y2 = map(int, t.bbox)
            cv2.rectangle(frame, (x1, y1), (x2, y2), t.color, 2)
            
            # Uncomment ini kalau mau lihat partikelnya
            # for p in t.particles:
            #     px1, py1, px2, py2 = map(int, p)
            #     pcx, pcy = (px1+px2)//2, (py1+py2)//2
            #     cv2.circle(frame, (pcx, pcy), 1, (0, 255, 255), -1)
            
            info_text = f"ID:{t.id} | {t.helmet_status} | Speed:{t.speed:.1f}"
            cv2.putText(frame, info_text, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, t.color, 2)

        cv2.putText(frame, f"Active Trackers: {len(trackers)}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        frame_display = resize_display(frame, target_width=TARGET_WIDTH)
        cv2.imshow("Multi-Object PSO Tracking", frame_display)
        
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()
    print("selesai")

if __name__ == "__main__":
    main()
    


Loading Model...
selesai


In [19]:
# ==============================================================================
# 1. KONFIGURASI (UBAH DISINI)
# ==============================================================================
VIDEO_PATH = r"test\14813939_1080_1920_30fps.mp4"       
MODEL_PATH = r"runs/detect/train2/weights/best.pt" 
TARGET_WIDTH = 360  # Ukuran tampilan

# --- SETTING DETEKSI ---
CONF_THRESHOLD = 0.25    # Keyakinan minimum
HELMET_DIST_THRESHOLD = 250 # Jarak maksimal kepala ke motor

# --- SPEEDOMETER ---
SPEED_MIN_THRESHOLD = 2.0 # Filter speed noise

# ==============================================================================
# 2. HELPER FUNCTIONS
# ==============================================================================
def resize_display(frame, target_width=TARGET_WIDTH):
    h, w = frame.shape[:2]
    if w < target_width: return frame
    scale = target_width / float(w)
    new_dim = (target_width, int(h * scale))
    return cv2.resize(frame, new_dim, interpolation=cv2.INTER_AREA)

def check_helmet_status(motor_box, all_yolo_boxes, class_names):
    mx1, my1, mx2, my2 = motor_box
    mc = ((mx1+mx2)//2, (my1+my2)//2)
    
    final_status = "Unknown"
    final_color = (0, 255, 255) # Kuning
    
    for box in all_yolo_boxes:
        cls_id = int(box.cls[0])
        label = class_names[cls_id]
        bx1, by1, bx2, by2 = map(int, box.xyxy[0])
        hc = ((bx1+bx2)//2, (by1+by2)//2)
        
        # Cek jarak Euclidean
        if math.dist(mc, hc) < HELMET_DIST_THRESHOLD: 
            if label in ['Helmet', 'helmet']:
                return "PAKAI HELM", (0, 255, 0) # Hijau
            elif label in ['No_helmet', 'no_helmet', 'head']:
                return "TIDAK PAKAI HELM", (0, 0, 255) # Merah
                
    return final_status, final_color

# ==============================================================================
# 3. FUNGSI UTAMA (YOLO NATIVE TRACKING)
# ==============================================================================
def run_yolo_native():
    print("Loading Model...")
    model = YOLO(MODEL_PATH)
    cap = cv2.VideoCapture(VIDEO_PATH)
    
    # Dictionary untuk menyimpan posisi sebelumnya berdasarkan ID
    # Format: { track_id: (center_x, center_y) }
    track_history = {}
    
    # Dictionary untuk menyimpan speed terakhir (biar ga flickering 0)
    speed_history = {}

    print(f"Memproses video: {VIDEO_PATH}")

    while True:
        start_time = time.time()
        ret, frame = cap.read()
        if not ret: break

        # 1. TRACKING BAWAAN YOLO (persist=True adalah kuncinya)
        # Ini menjalankan BoT-SORT/ByteTrack di background
        results = model.track(frame, persist=True, conf=CONF_THRESHOLD, verbose=False)[0]
        
        # 2. PROSES SETIAP OBJEK
        if results.boxes.id is not None: # Pastikan ada objek yang di-track
            
            # Ambil ID dan Box
            boxes = results.boxes.xyxy.cpu().numpy().astype(int)
            track_ids = results.boxes.id.cpu().numpy().astype(int)
            clss = results.boxes.cls.cpu().numpy().astype(int)
            
            for box, track_id, cls in zip(boxes, track_ids, clss):
                label = model.names[cls]
                
                # Kita hanya fokus tracking MOTOR
                if label in ['Motor', 'motorcycle', 'motorbike', 'scooter']:
                    x1, y1, x2, y2 = box
                    curr_center = ((x1+x2)//2, (y1+y2)//2)
                    
                    # A. Cek Status Helm (Logic Proximity sama dengan PSO)
                    status_txt, status_col = check_helmet_status(box, results.boxes, model.names)
                    
                    # B. Hitung Speed berdasarkan ID
                    speed = 0
                    if track_id in track_history:
                        prev_center = track_history[track_id]
                        dist = math.dist(prev_center, curr_center)
                        
                        # Filter noise speed
                        if dist > SPEED_MIN_THRESHOLD:
                            speed = dist
                        else:
                            speed = 0 # Anggap diam
                    
                    # Update History
                    track_history[track_id] = curr_center
                    speed_history[track_id] = speed
                    
                    # C. Visualisasi
                    # Gambar Kotak
                    cv2.rectangle(frame, (x1, y1), (x2, y2), status_col, 3)
                    
                    # Info Teks
                    last_speed = speed_history.get(track_id, 0)
                    cv2.putText(frame, f"{status_txt}", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, status_col, 2)
                    cv2.putText(frame, f"ID:{track_id} | Speed:{last_speed:.1f}", (x1, y1-30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

        # 3. HITUNG FPS SYSTEM
        end_time = time.time()
        fps = 1 / (end_time - start_time) if (end_time - start_time) > 0 else 0
        
        # Tampilkan Label Mode
        cv2.putText(frame, f"MODE: YOLO NATIVE (BENCHMARK)", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
        cv2.putText(frame, f"FPS: {fps:.1f}", (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # 4. TAMPILKAN VIDEO
        frame_display = resize_display(frame, target_width=TARGET_WIDTH)
        window_name = "YOLO Native Benchmark"
        cv2.imshow(window_name, frame_display)

        # --- LOGIKA STOP (Tombol Q atau Silang) ---
        key = cv2.waitKey(30) & 0xFF
        if key == ord('q'):
            print("Stop by user (Q).")
            break
        try:
            if cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE) < 1:
                print("Stop by user (X).")
                break
        except:
            break

    cap.release()
    cv2.destroyAllWindows()
    print("Selesai.")

if __name__ == "__main__":
    run_yolo_native()

Loading Model...
Memproses video: test\14813939_1080_1920_30fps.mp4
Stop by user (Q).
Selesai.
